# Работа 3

## Вступление

В этой лабораторной работе решаются задачи двух типов: одно уравнение $f(x)=0$ и система уравнений $F(\mathbf{x})=0$, где $\mathbf{x}=(x_1,\dots,x_n)$, а $F(\mathbf{x})=(f_1(\mathbf{x}),\dots,f_n(\mathbf{x}))$.
Хотим сравнить несколько численных методов: как они сходятся, какие условия им нужны и чем они отличаются.

Для одного уравнения используются методы:
0) половинное деление, 1) простая итерация, 2) Ньютон.

Для системы используются методы:
0) простая итерация, 1) Ньютон, 2) модифицированный Ньютон.

---

In [547]:
from sympy import Symbol
from sympy import diff
from sympy import lambdify
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations,
    implicit_multiplication_application,
    convert_xor,
)

_TRANSFORMS = standard_transformations + (implicit_multiplication_application, convert_xor)

In [548]:
tol = 1e-10
max_iter = 50

In [549]:
def build_numeric_function(expr_str, var_names):
  vars_syms = [Symbol(v) for v in var_names]
  local_dict = {name: sym for name, sym in zip(var_names, vars_syms)}

  expr = parse_expr(expr_str, local_dict=local_dict, transformations=_TRANSFORMS)
  f_raw = lambdify(vars_syms, expr, modules=("math", "sympy"))

  if len(var_names) == 1:

    def f(x):
      return float(f_raw(float(x)))

    return expr, f

  def f(vec):
    vals = [float(v) for v in vec]
    return float(f_raw(*vals))

  return expr, f

In [550]:
def numeric_derivative(f, x, h=1e-6):
  return (f(x + h) - f(x - h)) / (2.0 * h)


## Метод 0: половинное деление (для уравнения)

Задача: найти корень $f(x)=0$ на отрезке $[a,b]$. Метод работает, если $f(a)\cdot f(b)<0$, то есть на концах разные знаки.

1) берём середину $m=\frac{a+b}{2}$;
2) выбираем половину отрезка, где знак меняется;
3) повторяем, пока отрезок не станет меньше чем значение которое считаем за 0 (в коде - `tol`).

Можно выделить следующие преимущества:
- надёжный метод - если есть смена знака, корень точно есть в отрезке
- не требует производных

Однако, нельзя не сказать что метод сходится медленно и требует указание отрезка, на котором искать корень. Также не факт, что найденный корень будет единственным на отрезке

In [551]:
def bisect(f, a, b):
  fa = f(a)
  fb = f(b)

  if fa == 0.0:
      return a, 0
  if fb == 0.0:
      return b, 0
  if fa * fb > 0.0:
    print("bisect: изначальные значения имеют одинаковый знак! Не могу решить уравнение")
    return None, 0

  left = a
  right = b
  for it in range(max_iter):
    mid = 0.5 * (left + right)
    fm = f(mid)
    # fm == 0 || r - l == 0 => return "root found"
    if abs(fm) < tol or abs(right - left) < tol:
      return mid, it+1
    # пересекют прямую слева
    if fa * fm < 0.0:
      right = mid
      fb = fm
    # ... или справа
    else:
      left = mid
      fa = fm
  return mid, max_iter


---

## Метод 1: простая итерация (для уравнения)

Выбирается итерационная формула:
$$ \boxed{x_{k+1} = x_k - \lambda f(x_k)} $$

Метод заключается в том что мы с каждой новой итерацией мы все ближе и ближе к корню (см рис. - идем либо ступеньками либо по спирали).

Преимущества:
- простой алгоритм, который удобно программировать :-)
- не нужны производные

Недостатки:
- сходимость не гарантирована: важно выбрать $\lambda$ и начальное приближение (очень часто когда я тестировал, алгоритм не сходился, надо аккуратно подбирать начальные данные)
- обычно сходится заметно медленнее Ньютона
- при плохом $\lambda$ может расходиться

Заметним главное отличие от Ньютона: не нужна производная, метод более лёгкий, но менее мощный и более чувствительный.

<img src="./simple_iter.png"></img>

А далее, переходим к методу Ньютона...

In [552]:
def simple_iteration_from_f(f, x0, lam):
  x = float(x0)
  for it in range(1, max_iter + 1):
    x_next = x - lam * f(x)
    if abs(x_next - x) < tol:
      return x_next, it
    x = x_next
  return x, max_iter

---

## Метод 2: Ньютон (уравнение и система)

### Для уравнения
Формула:
$$ \boxed{x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}} $$

То есть на каждом шаге строится касательная к графику функции в точке $x_k$, и следующий $x_{k+1}$ берётся как пересечение касательной с осью $x$.

Преимущества:
- обычно быстро сходится, если начальная точка близка к решению;
- при хороших условиях скорость часто близка к квадратичной

Минусы:
- нужны производные $f'(x)$ (у меня считаются численно т к хотел сделать универсальный ввод уравнений);
- при плохом начальном приближении также может расходиться.

Главное отличие от простой итерации: Ньютон использует касательные чтобы локально аппроксимировать функцию, поэтому обычно даёт меньше итераций.

<img src="./newton.png"></img>

In [553]:
def newton(f, x0):
  x = float(x0)
  for it in range(1, max_iter + 1):
    fx = f(x)
    dfx = numeric_derivative(f, x)
    if dfx == 0.0:
      return x, it, False

    x_next = x - fx / dfx
    if abs(x_next - x) < tol:
      return x_next, it, True
    x = x_next
  return x, max_iter, True

---

#### Выбираем что решать и как:

In [554]:
print("Решаем f(x) = 0")
expr_str = input("Введите f(x) (например: x^3 - x - 1): ").strip()

expr, f = build_numeric_function(expr_str, ["x"])

a = float(input("a: ").strip())
b = float(input("b: ").strip())
root, it = bisect(f, a, b)
ok = True
print("Половинное деление:")
print(f"root = {root}")
if root is not None:
  print(f"f(root) = {f(root)}")
print(f"iters = {it}")
print(f"ok = {ok}")

x0 = float(input("x0: ").strip())
lam_in = input("λ: ").strip()
lam = float(lam_in)
root, it= simple_iteration_from_f(f, x0, lam)
print("Простая итерация:")
print(f"root = {root}")
if root is not None:
  print(f"f(root) = {f(root)}")
print(f"iters = {it}")
print(f"ok = {ok}")

x0 = float(input("x0: ").strip())
root, it, ok = newton(f, x0)
print("Метод Ньютона:")
print(f"root = {root}")
if root is not None:
  print(f"f(root) = {f(root)}")
print(f"iters = {it}")
print(f"ok = {ok}")

Решаем f(x) = 0
Половинное деление:
root = -0.9999999999781721
f(root) = -4.3655745685100555e-11
iters = 36
ok = True
Простая итерация:
root = 0.9999662577032996
f(root) = -6.748345485818597e-05
iters = 50
ok = True
Метод Ньютона:
root = 1.0
f(root) = 0.0
iters = 1
ok = True


---
---

## Решение систем уравнений

In [555]:
def build_numeric_system(eqs_str, var_names):
  vars_syms = [Symbol(v) for v in var_names]
  local_dict = {name: sym for name, sym in zip(var_names, vars_syms)}

  eq_strs = [s.strip() for s in eqs_str.split(";") if s.strip()]
  assert eq_strs, "Система должна быть не пустой"

  exprs = []
  for s in eq_strs:
    exprs.append(parse_expr(s, local_dict=local_dict, transformations=_TRANSFORMS))

  F_raw = []
  for expr in exprs:
    F_raw.append(lambdify(vars_syms, expr, modules=("math", "sympy")))

  def F(vec):
    vals = [float(v) for v in vec]
    out = []
    for f in F_raw:
      out.append(float(f(*vals)))
    return out

  return exprs, F


In [556]:
def build_numeric_jacobian(exprs, var_names):
  vars_syms = [Symbol(v) for v in var_names]

  J_expr = []
  for expr in exprs:
    row = []
    for v in vars_syms:
      row.append(diff(expr, v))
    J_expr.append(row)

  J_raw = []
  for i in range(len(exprs)):
    row_f = []
    for j in range(len(var_names)):
      row_f.append(lambdify(vars_syms, J_expr[i][j], modules=("math", "sympy")))
    J_raw.append(row_f)

  def J(vec):
    vals = [float(v) for v in vec]
    out = []
    for i in range(len(exprs)):
      row = []
      for j in range(len(var_names)):
        row.append(float(J_raw[i][j](*vals)))
      out.append(row)
    return out

  return J_expr, J


In [557]:
def norm_inf(vec):
  return max(abs(v) for v in vec)

In [558]:
def solve_linear_gauss(A, b):
  n = len(A)
  # [A|b]
  M = [] # копия
  for i in range(n):
    row = []
    for j in range(n):
      row.append(float(A[i][j]))
    row.append(float(b[i]))
    M.append(row)

  # Прямой ход
  for k in range(n - 1):
    akk = M[k][k]
    if akk == 0.0:
      return None

    for i in range(k + 1, n):
      factor = M[i][k] / akk
      for j in range(k, n + 1):
        M[i][j] = M[i][j] - factor * M[k][j]

  # Обратный ход
  x = [0.0] * n
  for i in range(n - 1, -1, -1):
    aii = M[i][i]
    if aii == 0.0:
      return None

    s = M[i][n]
    for j in range(i + 1, n):
      s = s - M[i][j] * x[j]

    x[i] = s / aii

  return x

## Метод 1: простая итерация (для системы)

Аналогично:
$$ \boxed{\mathbf{x}_{k+1} = \mathbf{x}_k - \lambda F(\mathbf{x}_k)} $$
где $\lambda$ — одно число для всех компонент.

Плюсы:
- простой;
- не нужен якобиан (см далее);

Из минусов: опять часто расходится на тестах

In [559]:
def simple_iteration_system(F, x0, lam):
  x = [float(v) for v in x0]
  for it in range(1, max_iter + 1):
    fx = F(x)
    x_next = [x[i] - lam * fx[i] for i in range(len(x))]
    if norm_inf([x_next[i] - x[i] for i in range(len(x))]) < tol:
      return x_next, it, True
    x = x_next
  return x, max_iter, True


---

## Метод 2: Ньютон (для система)

$$ \mathbf{x}_{k+1} = \mathbf{x}_k - J^{-1}(\mathbf{x}_k) F(\mathbf{x}_k), \quad J(\mathbf{x}_k) (\mathbf{x}_{k+1} - \mathbf{x}_k) = -F(\mathbf{x}_k) =>$$
$$ \boxed{J(\mathbf{x}_k)\,\Delta \mathbf{x}_k = -F(\mathbf{x}_k)} $$

Нужно решать эту линейную систему на каждом шаге,
где $J(\mathbf{x})$ — матрица Якоби:
$J_{ij}=\frac{\partial f_i}{\partial x_j}$.

В целом, идея та же, но производная меняется на якобиан

Плюсы:
- быстро сходится
- скорость часто близка к квадратичной.

Минусы:
- вычисление Якобиана достаточно дорогое
- требуется решать линейную систему (в коде это Гаусс);
- все еще возможна расходимость

Вычисление якобиана дает большой оверхед, хотелось бы избежать этого... (см модифицированный метод)


In [560]:
def newton_system(F, J, x0):
  x = [float(v) for v in x0]
  for it in range(1, max_iter + 1):
    fx = F(x)
    if norm_inf(fx) < tol:
      return x, it, True

    A = J(x)
    b = [-v for v in fx]
    delta = solve_linear_gauss(A, b)
    if delta is None:
      return x, it, False

    x_next = [x[i] + delta[i] for i in range(len(x))]
    if norm_inf([x_next[i] - x[i] for i in range(len(x))]) < tol:
      return x_next, it, True
    x = x_next

  return x, max_iter, True


---

## Метод 3: модифицированный Ньютон (только для системы)

В этом методе Якобиан фиксируется один раз в начальной точке:
$J(\mathbf{x}_0)$.

Дальше на каждом шаге решается:
$J(\mathbf{x}_0)\,\Delta = -F(\mathbf{x}_k)$,
$\mathbf{x}_{k+1} = \mathbf{x}_k + \Delta$.

Преимущества:
- дешевле: Якобиан не пересчитывается каждый шаг
- проще и быстрее по времени на итерацию (особенно для больших систем).

Минусы:
- может хуже сходиться, если Якобиан сильно меняется по области.


In [561]:
def modified_newton_system(F, J0, x0):
  x = [float(v) for v in x0]
  for it in range(1, max_iter + 1):
    fx = F(x)
    if norm_inf(fx) < tol:
      return x, it, True

    b = [-v for v in fx]
    delta = solve_linear_gauss(J0, b)
    if delta is None:
      return x, it, False

    x_next = [x[i] + delta[i] for i in range(len(x))]
    if norm_inf([x_next[i] - x[i] for i in range(len(x))]) < tol:
      return x_next, it, True
    x = x_next

  return x, max_iter, True


---

#### Решаем саму систему

In [565]:
print("Введите систему: F(x) = 0")
print("Введите уравнения через ';' (пример: x^2 + y^2 - 1; x - y)")
eqs_str = input("Система: ").strip()

vars_str = input("Переменные через пробел (пример: x y): ").strip()
var_names = [v.strip() for v in vars_str.split() if v.strip()]
if not var_names:
    var_names = ["x", "y"]

exprs, F = build_numeric_system(eqs_str, var_names)
_, J = build_numeric_jacobian(exprs, var_names)

x0 = []
print("Введите Начальное приближение:")
for v in var_names:
    x0.append(float(input(f"{v}0: ")))

lam = 0.1 # float(input("λ (например 0.1): ").strip())
sol, it, ok = simple_iteration_system(F, x0, lam)
print("Простая итерация:")
for i, v in enumerate(var_names):
  print(f"{v} = {sol[i]}")
print(f"F(sol) = {F(sol)}")
print(f"iters = {it}")
print(f"ok = {ok}")

sol, it, ok = newton_system(F, J, x0)
print("Метод Ньютона:")
for i, v in enumerate(var_names):
  print(f"{v} = {sol[i]}")
print(f"F(sol) = {F(sol)}")
print(f"iters = {it}")
print(f"ok = {ok}")

J0 = J(x0)
sol, it, ok = modified_newton_system(F, J0, x0)
print("Модифицированный метод Ньютона:")
for i, v in enumerate(var_names):
  print(f"{v} = {sol[i]}")
print(f"F(sol) = {F(sol)}")
print(f"iters = {it}")
print(f"ok = {ok}")


Введите систему: F(x) = 0
Введите уравнения через ';' (пример: x^2 + y^2 - 1; x - y)
Введите Начальное приближение:
Простая итерация:
x = -1113.8698562135673
y = 2697.774494642983
F(sol) = [1580.9046384294156, -3816.6443508565503]
iters = 50
ok = True
Метод Ньютона:
x = 4.0
y = -1.0
F(sol) = [0.0, 0.0]
iters = 2
ok = True
Модифицированный метод Ньютона:
x = 4.0
y = -1.0
F(sol) = [0.0, 0.0]
iters = 2
ok = True


## Результаты

В результате работы были реализованы методы для уравнений и систем и проведены вычисления для выбранных примеров.

Общее наблюдение по методам:
- половинное деление (для уравнения) почти всегда сходится, но требует отрезок $[a,b]$ со сменой знака и даёт много итераций;
- простая итерация для уравнения и системы сильно зависит от параметра $\lambda$ и начального приближения;
- Ньютон обычно требует меньше итераций, но нуждается в производных и хорошей стартовой точке;
- модифицированный Ньютон для системы часто занимает промежуточное место: итераций больше, чем у Ньютона, зато одна итерация дешевле.

---

## Обсуждение результатов

Методы в этой лабораторной можно поделить на надёжные и быстрые.

- половинное деление — самый надёжный, но медленный;
- простая итерация — простой, но чувствителен к $\lambda$;
- Ньютон — быстрый по числу итераций, но требует вычисления производной (якобиана).

- модифицированный Ньютон — якобиан считаем всего 1 раз, но из-за этого больше итераций.

В целом выбор метода зависит от того, что важнее: гарантированная сходимостьили скорость (как у Ньютона) и насколько удобно вычислять производные/Якобиан.
